# 2. 데이터 준비 및 전처리

이 챕터에서는 MovieLens 데이터셋을 다운로드하고 전처리하는 방법을 다룹니다. 데이터 로딩, 훈련/테스트 분할, 그리고 데이터 탐색을 포함합니다.

## 2.1 필수 라이브러리 불러오기

In [1]:
import os
import sys
import io
import zipfile
import requests
import warnings
warnings.filterwarnings('ignore')

# 상위 디렉토리 경로를 시스템 경로에 추가하여 utils 모듈을 import할 수 있게 함
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Optional

# utils 모듈에서 필요한 클래스 import
from utils.models import Dataset

/Users/al02614027/Documents/workspace/demo/TIL-NEW/recommend/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 2.2 데이터 로더 클래스 구현

In [ ]:
class DataLoader:
    def __init__(self, num_users: Optional[int] = 100, num_test_items: int = 5):
        self.num_users = num_users
        self.num_test_items = num_test_items
        self.data_dir = "./ml-latest-small"
        self.dataset_url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

    def _download_and_extract(self):
        if not os.path.exists(self.data_dir):
            print(f"Downloading dataset from {self.dataset_url}...")
            r = requests.get(self.dataset_url, timeout=60)
            r.raise_for_status()
            z = zipfile.ZipFile(io.BytesIO(r.content))
            z.extractall(".")
            print("Download and extraction complete!")
        else:
            print(f"Dataset already exists in {self.data_dir}")

    def load(self) -> Dataset:
        self._download_and_extract()
        ratings, movies = self._load_dataframes()
        train, test = self._split_data(ratings)

        test_user2items = (
            test[test.rating >= 4].groupby("userId")["movieId"].apply(list).to_dict()
        )
        return Dataset(train, test, test_user2items, movies)

    def _split_data(self, df: pd.DataFrame):
        # 유저별 최근 5개를 Test
        df = df.copy()
        df["rating_order"] = df.groupby("userId")["timestamp"].rank(ascending=False, method="first")
        train = df[df["rating_order"] > self.num_test_items]
        test  = df[df["rating_order"] <= self.num_test_items]
        return train, test

    def _load_dataframes(self):
        ratings = pd.read_csv(os.path.join(self.data_dir, "ratings.csv"))
        movies  = pd.read_csv(os.path.join(self.data_dir, "movies.csv"))
        if self.num_users is not None:
            valid_users = sorted(ratings.userId.unique())[:self.num_users]
            ratings = ratings[ratings.userId.isin(valid_users)]
        return ratings, movies

## 2.3 데이터 로드 및 탐색

In [ ]:
# 데이터 로드
loader = DataLoader(num_users=100)  # 테스트를 위해 100명으로 제한
dataset = loader.load()

# 데이터 탐색
print(f"훈련 데이터 형태: {dataset.train.shape}")
print(f"테스트 데이터 형태: {dataset.test.shape}")
print(f"고유 사용자 수: {dataset.train.userId.nunique()}")
print(f"고유 영화 수: {dataset.train.movieId.nunique()}")

## 2.4 영화 데이터 샘플 확인

In [ ]:
# 영화 데이터 샘플 확인
dataset.item_content.head()